# MSDS 7331 - Mini-Project: SVM&LR Classification
## Team: Dineen Parker, Dale Legband, Ryan Shuhart
collaboration site: https://github.com/rlshuhart/MSDS7331_Mini-Lab

### Log to capture iterations of findings and ideas for feature selection and analysis

1. First attempt of logistic model revealed need to one-hot encode categorical variables
2. After numerically encoding categorical variables, the logistic model more or less predicted all to not default.
3. Changeing penalty on logistic model from l2 to l1 improved the model by about 3 percentage points.
4. Changing class_weight='balanced' negatively impacted model
5. Changing C to 100 had no impact
5. Changing C to .0001 had small negative impact

## SVM and Logistic Regression Modeling  
* [50 points] Create a logistic regression model and a support vector machine model for the classiﬁcation task involved with your dataset. Assess how well each model performs (use 80/20 training/testing split for your data). Adjust parameters of the models to make them more accurate. If your dataset size requires the use of stochastic gradient descent, then linear kernel only is ﬁne to use. 
* [10 points] Discuss the advantages of each model for each classiﬁcation task. Does one type of model offer superior performance over another in terms of prediction accuracy? In terms of training time or efﬁciency? Explain in detail.  
* [30 points] Use the weights from logistic regression to interpret the importance of different features for each classiﬁcation task. Explain your interpretation in detail. Why do you think some variables are more important? 
* [10 points] Look at the chosen support vectors for the classiﬁcation task. Do these provide any insight into the data? Explain

In [1]:
### Modules Used ###

# Data manipulation: pandas and numpy
import pandas as pd
import numpy as np

# Visualization: seaborn and matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Read in preprocessed data from previous assignment

In [2]:
# Random but useful fact: if the first record in a csv file is 
# "ID" than Excel will interpret it as a SYLK file
# https://annalear.ca/2010/06/10/why-excel-thinks-your-csv-is-a-sylk/

data_url = "https://raw.githubusercontent.com/rlshuhart/MSDS7331_Project_1/master/cc_data/cc_data_processed.csv"
#data_url = "../MSDS7331_Project_1/cc_data/cc_data_processed.csv"

cc_data = pd.read_csv(data_url, index_col='ID')

In [3]:
print(cc_data.info())
print()
print(cc_data.columns)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 1 to 30000
Data columns (total 27 columns):
LIMIT_BAL                     30000 non-null int64
SEX                           30000 non-null object
EDUCATION                     30000 non-null object
MARRIAGE                      30000 non-null object
AGE                           30000 non-null int64
PAY_0_Sept                    30000 non-null int64
PAY_2_Aug                     30000 non-null int64
PAY_3_July                    30000 non-null int64
PAY_4_June                    30000 non-null int64
PAY_5_May                     30000 non-null int64
PAY_6_April                   30000 non-null int64
BILL_AMT1_Sept                30000 non-null int64
BILL_AMT2_Aug                 30000 non-null int64
BILL_AMT3_July                30000 non-null int64
BILL_AMT4_June                30000 non-null int64
BILL_AMT5_May                 30000 non-null int64
BILL_AMT6_April               30000 non-null int64
PAY_AMT1_Sept        

# Additional Preprocessing of data

### One hot encoding of categorical variables

In [4]:
# makeing copy of data to retain original input for now. 
# Have to add .copy() or any change will reflect in the original too.
cc_data_processed = cc_data.copy()

cats = ['SEX', 'EDUCATION', 'MARRIAGE','AGE_range', 'LIMIT_BAL_range']

# SEX convert to isMale
cc_data_processed['isMale'] = cc_data['SEX']=='Male'

# Two options for Education: 
# If treating ordinal then keep single columnn but label in numerical order
# if nominal then create dummy variables.
# Trying ordinal first with 'Others' as the highest. Previous exploration
# showed this group have better rates of default which continued the trend
# of more education means less likely to default:
cc_data_processed['EDUCATION'] = cc_data['EDUCATION'].apply({'High School':0,
                                                             'University':1,
                                                             'Graduate School':2, 
                                                             'Others':3}.get)

# Marriage is nominal category therefore converting to dummies

cc_data_processed = pd.concat((cc_data_processed, 
                               pd.get_dummies(cc_data['MARRIAGE'], prefix='Marriage')), 
                               axis=1)

# Age range is ordinal category:
cc_data_processed['AGE_range'] = cc_data['AGE_range'].apply({'21-30':0,
                                                             '31-40':1,
                                                             '41-50':2, 
                                                             '51-60':3,
                                                             '61-70':4,
                                                             '71-80+':5}.get)
# Limit_Bal_Range is ordingal
cc_data_processed['LIMIT_BAL_range'] = cc_data['LIMIT_BAL_range'].apply({'0-250k':0,
                                                                         '>250k-500k':1,
                                                                         '>500k-750k':2,
                                                                         '>750k':3}.get)

# Drop unused features:
# 'Default_Yes_No' - Used for visualization labeling only and same as 'default payment next month'
# 'AGE' - Using Age_range instead, may bring back
cc_data_processed.drop(['Default_Yes_No','AGE', 'SEX','MARRIAGE', 'LIMIT_BAL'], axis=1, inplace=True)
print(cc_data_processed.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 1 to 30000
Data columns (total 26 columns):
EDUCATION                     30000 non-null int64
PAY_0_Sept                    30000 non-null int64
PAY_2_Aug                     30000 non-null int64
PAY_3_July                    30000 non-null int64
PAY_4_June                    30000 non-null int64
PAY_5_May                     30000 non-null int64
PAY_6_April                   30000 non-null int64
BILL_AMT1_Sept                30000 non-null int64
BILL_AMT2_Aug                 30000 non-null int64
BILL_AMT3_July                30000 non-null int64
BILL_AMT4_June                30000 non-null int64
BILL_AMT5_May                 30000 non-null int64
BILL_AMT6_April               30000 non-null int64
PAY_AMT1_Sept                 30000 non-null int64
PAY_AMT2_Aug                  30000 non-null int64
PAY_AMT3_July                 30000 non-null int64
PAY_AMT4_June                 30000 non-null int64
PAY_AMT5_May            

### Convert data to numpy arrays

In [5]:
cc_data_target = cc_data_processed['default payment next month'].values
cc_data_Xs = cc_data_processed.drop('default payment next month', axis='columns').values

# Prediction Models

In [6]:
# Scikit Modules
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics as mt
from sklearn.linear_model import LogisticRegression

In [7]:
# Sample Default Rate
sample_d_rt = (1 - cc_data_target.sum() / len(cc_data_target))*100
print("The sample default rate is: %.1f%%"  % sample_d_rt)

# The sample default rate is the same as predicting all zeros (or no defaults)
all_zeros = np.zeros(len(cc_data_target))
all_zero_score = mt.accuracy_score(cc_data_target,all_zeros)*100
print("The models should have an accuracy better than predicting all non-defaults of %.1f%%"  % all_zero_score)

The sample default rate is: 77.9%
The models should have an accuracy better than predicting all non-defaults of 77.9%


## Logistic Regression

In [8]:
model = LogisticRegression(penalty='l2', C=1.0, class_weight=None)

def classify(model, X, y):
    accuracies = cross_val_score(model,X,y=y)
    predicted = cross_val_predict(model,X,y=y)
    print(accuracies)
    return(predicted)

p = classify(model, cc_data_Xs, cc_data_target)

#below compares how close to all zero prediction
mt.accuracy_score(p,all_zeros)*100

[ 0.7799  0.7827  0.7826]


99.333333333333329

In [9]:
#changed penalty to 'l1'
model = LogisticRegression(penalty='l1', C=1.0, class_weight=None)
p = classify(model, cc_data_Xs, cc_data_target)

#below compares how close to all zero prediction
mt.accuracy_score(p,all_zeros)*100

[ 0.806   0.8135  0.8079]


92.63333333333334

In [10]:
#changed class_weight to 'balanced'
model = LogisticRegression(penalty='l1', C=1.0, class_weight='balanced')
p = classify(model, cc_data_Xs, cc_data_target)

#below compares how close to all zero prediction
mt.accuracy_score(p,all_zeros)*100

[ 0.683   0.6775  0.7151]


63.339999999999996

In [11]:
# Increased C to 1000
model = LogisticRegression(penalty='l1', C=1000.0, class_weight=None)
p = classify(model, cc_data_Xs, cc_data_target)

#below compares how close to all zero prediction
mt.accuracy_score(p,all_zeros)*100

[ 0.806   0.8135  0.808 ]


92.63333333333334

In [12]:
# Decreased C to .001
model = LogisticRegression(penalty='l1', C=0.001, class_weight=None)
p = classify(model, cc_data_Xs, cc_data_target)

#below compares how close to all zero prediction
mt.accuracy_score(p,all_zeros)*100

[ 0.7969  0.8051  0.797 ]


94.416666666666671

# Intrepretting weights

In [13]:
#Weights are not to scale 
weights = cc_data_Xs.T 
#variable_names = cc_data.columns <- cc_data is the old data frame
variable_names = cc_data_processed.columns
for coef, name in zip(weights,variable_names):
    print(name, 'has weight of', coef[0])

EDUCATION has weight of 1
PAY_0_Sept has weight of 2
PAY_2_Aug has weight of 2
PAY_3_July has weight of -1
PAY_4_June has weight of -1
PAY_5_May has weight of -2
PAY_6_April has weight of -2
BILL_AMT1_Sept has weight of 3913
BILL_AMT2_Aug has weight of 3102
BILL_AMT3_July has weight of 689
BILL_AMT4_June has weight of 0
BILL_AMT5_May has weight of 0
BILL_AMT6_April has weight of 0
PAY_AMT1_Sept has weight of 0
PAY_AMT2_Aug has weight of 689
PAY_AMT3_July has weight of 0
PAY_AMT4_June has weight of 0
PAY_AMT5_May has weight of 0
PAY_AMT6_April has weight of 0
default payment next month has weight of 0
AGE_range has weight of 0
LIMIT_BAL_range has weight of False
isMale has weight of 1.0
Marriage_Married has weight of 0.0
Marriage_Others has weight of 0.0


In [14]:
#Weights are to scale 
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

scl_obj = StandardScaler()
scl_obj.fit(cc_data_Xs)
cc_data_Xs_scaled = scl_obj.transform(cc_data_Xs)

model = LogisticRegression(penalty='l2', C=1.0, class_weight=None)

def classify(model, X,y):
    accuracies = cross_val_score(model,X,y=y) 
    print(accuracies)
classify(model, cc_data_Xs_scaled, cc_data_target)

#error, I think there is an error because of the boolean on PAY_AMT5_May*****************
## the accuracy changed but the weights didn't
zip_vars = zip(cc_data_Xs.T,cc_data_processed.columns) 
for coef, name in zip_vars:
    print(name, 'has weight of', coef[0])


C:\Users\ryan.shuhart\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\ryan.shuhart\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


[ 0.8059  0.8134  0.808 ]
EDUCATION has weight of 1
PAY_0_Sept has weight of 2
PAY_2_Aug has weight of 2
PAY_3_July has weight of -1
PAY_4_June has weight of -1
PAY_5_May has weight of -2
PAY_6_April has weight of -2
BILL_AMT1_Sept has weight of 3913
BILL_AMT2_Aug has weight of 3102
BILL_AMT3_July has weight of 689
BILL_AMT4_June has weight of 0
BILL_AMT5_May has weight of 0
BILL_AMT6_April has weight of 0
PAY_AMT1_Sept has weight of 0
PAY_AMT2_Aug has weight of 689
PAY_AMT3_July has weight of 0
PAY_AMT4_June has weight of 0
PAY_AMT5_May has weight of 0
PAY_AMT6_April has weight of 0
default payment next month has weight of 0
AGE_range has weight of 0
LIMIT_BAL_range has weight of False
isMale has weight of 1.0
Marriage_Married has weight of 0.0
Marriage_Others has weight of 0.0
